In [1]:
#import the correct packages
import pandas as pd

In [ ]:
# read the csv file from your results. Change the path to YOUR results. 
data_run1_all = pd.read_csv("/Users/jstanton/Desktop/NS005_Sorghum.csv")

In [ ]:
# read your barcode file, this will help you get readable data out
run1_barcodes = pd.read_csv("/Users/jstanton/Desktop/ns_005_barcodes.csv")

In [ ]:
#get rid of all the extra columns that you don't need. Change this if there is something you are particulalry interested in. Otherwise, this will make your file much smaller. 
data_run1_all.drop(['camera','imgtype','zoom','exposure','gain','frame','lifter','id','treatment','velocity','cartag','measurementlabel','other','image','sample','rotation'], inplace=True, axis=1)

In [ ]:
#make sure the time is converted properly by making a new column called time2
data_run1_all['time2'] = pd.to_datetime(data_run1_all['timestamp'])

In [ ]:
#take a look at the first few rows of data so that they contain what you are interested in
data_run1_all.head()

In [ ]:
#convert the time to just the date if you don't care when in the day the photo was taken.
data_run1_all['date'] = data_run1_all['time2'].dt.date

In [ ]:
#get rid of your extra time columns
data_run1_all.drop(['timestamp','time2'], inplace=True, axis=1)

In [ ]:
#take a look at the first few rows of data so that they contain what you are interested in
data_run1_all.head()

In [ ]:
# select rows containing height
data_run1_height = data_run1_all[data_run1_all['trait'].str.contains('height_above_reference')]

In [ ]:
#take a look at the first few rows of data so that they contain what you are interested in
data_run1_height.head()

In [ ]:
#convert height in pixels ot height in cm. A chip on the color card in the plant image is 1.2 cm - yours is also 1.2 cm (NOT the giant color card. 
#For my images, the color chip size was 38 pixels (this depends on your zoom). You got this from your notebook and saved it for later in the tutorial, so refer back to
#when you found color chip size for this pixel value! 

data_run1_height["height_cm"] = data_run1_height["value"] * 1.2/38

In [ ]:
#get rid of the extra columns, you only want height in cm

data_run1_height.drop(['trait','value','label'], inplace=True, axis=1)

In [ ]:
#find the max height between the two side pictures of the same plant - you only want one height for the whole plant, so take the maximum one. 
#You can take average if you would like instead. This leaves you with a single measurement per plant per day. 

height = data_run1_height.groupby(['date','barcode'], as_index=False).max(['height_cm'])
height.head()

In [ ]:
#check how big this dataframe is. Is it half the size it was before you took only the max height?
len(height)

In [ ]:
#merge your data with your barcodes so that your data is properly labeled with genotype, treatment, etc.

height_merge = pd.merge(height, run1_barcodes, how = "left", on = ['barcode'])

In [ ]:
#get rid of empty rows

height_merge = height_merge.dropna(axis = 'rows')

In [ ]:
#take a look at the first few rows of data so that they contain what you are interested in

height_merge.head()

In [ ]:
#check the length again, it should be close to what it was before you got rid of empty rows. If not, something didn't merge properly

len(height_merge)

In [ ]:
#save the resulting file to a csv for downstream analysis in R

height_merge.to_csv('ns_005_heights.csv')